# Install Libaries

In [1]:
!pip install langchain
!pip install langchain-openai
!pip install langchain-community

# Enter Open AI API key 

In [3]:
from getpass import getpass
OPENAPI_KEY=getpass("enter your api key here : ")

enter your api key here :  ········


# Set Environment Variables

In [5]:
import os 
os.environ["OPENAI_API_KEY"]=OPENAPI_KEY

# Load necessary Dependcies and Chatgpt LLM

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# LLM MODELS

In [9]:
chatgpt=ChatOpenAI(model_name="gpt-4",temperature=0.9)

## Access the IT Support queue data

In [11]:
it_support_queue=["Näo consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha. " ,
                    "Ho problemi a stampare i documenti da remoto. II lavoro non viene inviato alla stampante di rete. " ,
                  "我无法远程打印文档。作业未发送至网络打印机。",
                    "fl He Mory BOhTY•1 B CHCTemy yqeTa BpeMeHV1, noRBJIReTcg c0061L1eHb•1e 06 OUJM6Ke. MHe HY*Ha n0MOU1b." ,
                    "Internet badlantlm cok yavas ve bazen tamamen kesiliyor. Yardlm eder misiniz?" ,
                    "He mory YCTaHOBMTb 06HOBneHMe 6e30nacHocTh. r10RBngeTcg KOA OLUV16KH. 110MorhTe, noxanyicra."]
#give the output using JSON output paas

# Define Output Parser

In [13]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel , Field


class itsupportresponse(BaseModel):
    originalmsg:str =Field(Description="This is orginal customer IT support query message ")
    originallang:str =Field(Description="Detected language of the customer e.g. spanish ")
    categorymsg:str =Field(Description="1 to 2 words describing the category of the message ")
    transmsg:str =Field(Description="Translated customer IT support query language in english ")
    response:str =Field(Description="Response to the customer in originallang ")
    transresponse:str =Field(Description="Respnse to the customer in english")

parser=JsonOutputParser(pydantic_object=itsupportresponse)

C:\Users\KIIT\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


# create the input prompt for the LLM 

In [31]:
prompt_text = """
Act as Information Technology (IT) customer support agent.
For the support message mentioned below,
use the following output format while generating the output response:
{format_instruction}

Customer IT support message:
{it_support_msg}
"""

prompt = PromptTemplate(
    template=prompt_text,
    input_variables=["it_support_msg"],
    partial_variables={"format_instruction": parser.get_format_instructions()}
)


# create LCEL LLM chain

In [33]:
llm_chain=(prompt|chatgpt|parser)

In [35]:
it_support_queue=["Näo consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha. " ,
                    "Ho problemi a stampare i documenti da remoto. II lavoro non viene inviato alla stampante di rete. " ,
                  "我无法远程打印文档。作业未发送至网络打印机。",
                    "fl He Mory BOhTY•1 B CHCTemy yqeTa BpeMeHV1, noRBJIReTcg c0061L1eHb•1e 06 OUJM6Ke. MHe HY*Ha n0MOU1b." ,
                    "Internet badlantlm cok yavas ve bazen tamamen kesiliyor. Yardlm eder misiniz?" ,
                    "He mory YCTaHOBMTb 06HOBneHMe 6e30nacHocTh. r10RBngeTcg KOA OLUV16KH. 110MorhTe, noxanyicra."]
formatted_msgs=[{"it_support_msg": msg} for msg in it_support_queue ]

In [37]:
responses=llm_chain.map().invoke(formatted_msgs)

In [49]:
print(responses)

[{'originalmsg': 'Näo consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.', 'originallang': 'Portuguese', 'categorymsg': 'Sync issue', 'transmsg': "I can't sync my contacts with the phone. I always receive a failure message.", 'response': 'Por favor, tente reiniciar o seu telefone e tente sincronizar novamente. Se o problema persistir, por favor, verifique se a sua conta está corretamente configurada no seu telefone.', 'transresponse': 'Please try restarting your phone and sync again. If the problem persists, please check if your account is properly set up on your phone.'}, {'originalmsg': 'Ho problemi a stampare i documenti da remoto. II lavoro non viene inviato alla stampante di rete.', 'originallang': 'Italian', 'categorymsg': 'Printer Issue', 'transmsg': "I'm having trouble printing documents remotely. The job is not sent to the network printer.", 'response': 'Assicurati che la tua stampante sia connessa alla stessa rete del tuo dispositivo. Se il